In [6]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


In [7]:
def train_different_classes(model_path, tset, vset, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        # print(i)
        acc, balanced_acc = train_end_to_end(model_path, tset, vset, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

In [8]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
    
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


logo = LeaveOneGroupOut()
logo.get_n_splits(x_vals, y_vals, groups)

result_dict = {}
# dtype=torch.int32
for train_index, test_index in logo.split(x_vals, y_vals, groups):
    group_num = groups[test_index][0]
    print("Leaving out mouse number:", group_num)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    accuracy, balanced_accuracy = train_different_classes("TS_stagernet.pth", training_set, test_set, 50, num_samples)
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)


Loading Data
Processing MouseCKA1_030515_HCOFTS
removed 0 unknown entries
Processing MouseCKA1_030615_HCOFTS
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022815_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_070114_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_070114_HCOFTS
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070314_HCOFTS
removed 0 unknown entries
Processing 

Exception ignored in: <function _releaseLock at 0x7ffa8fa98950>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


RuntimeError: DataLoader worker (pid(s) 96846) exited unexpectedly

In [4]:
gc.collect()

60

In [5]:
RP_vals = train_different_classes("RP_stagernet.pth", training_set, validation_set, 100, num_samples)

1
len of the dataloader is: 1
Start Training
[0.5416666666666666]
10
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888]
None
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888, 0.8611111111111112]
